### Calling LLMs

In [2]:
from langchain.llms import OpenAI
import os 

In [9]:
llm = OpenAI(openai_api_key=os.environ["OPEN_API_KEY"], temperature = 0.6)

In [10]:
text="What is the capital of Turkey?"
print(llm.invoke(text))



The capital of Turkey is Ankara.


In [14]:
from langchain import HuggingFaceHub
llm_hugggingface = HuggingFaceHub(repo_id = "google/flan-t5-large", model_kwargs ={"temperature":0,"max_length":64})

In [15]:
output = llm_hugggingface.predict("What is the capital of Russia?")
print(output)

moscow


In [17]:
output = llm_hugggingface.predict("please write a short poem about AI")
print(output)

i love the way ai looks at me i love the way ai looks at me i love the way ai looks at me i love the way ai looks at me i love the way ai looks at me i love the way ai


### Prompt Templates and LLM Chain

In [20]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=['country'], 
                                 template= "Tell me the capital of {country}")
prompt_template.format(country = "Turkey")

'Tell me the capital of Turkey'

In [23]:
from langchain.chains import LLMChain
chain = LLMChain(llm = llm, prompt = prompt_template)
chain.run("Turkey")

'\n\nThe capital of Turkey is Ankara.'

### Combining Multiple Chains Using Simple Sequential Chain

In [27]:
capital_template = PromptTemplate(input_variables=['country'],
                               template = "Tell me the capital of {country}")
capital_chain = LLMChain(llm = llm, prompt = capital_template)

famous_template = PromptTemplate(input_variables=['capital'],
                                 template = "Give me some nice places to visit in {capital}")

In [28]:
famous_chain = LLMChain(llm = llm, prompt= famous_template)

In [29]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [capital_chain,famous_chain])
chain.run("Turkey")

" Here are some nice places to visit in Ankara:\n\n1. Anıtkabir: This is the mausoleum of Mustafa Kemal Atatürk, the founder of modern Turkey. It is a grand structure with beautiful gardens and is a must-visit for anyone interested in Turkish history.\n\n2. Kızılay Square: This is the heart of Ankara and a popular meeting place for locals. It is surrounded by shops, restaurants, and cafes, making it a great spot for people-watching and experiencing the city's vibrant atmosphere.\n\n3. Ankara Castle: Located on a hilltop, this medieval castle offers stunning views of the city. It is also home to several museums, including the Museum of Anatolian Civilizations, which displays artifacts from the region's ancient civilizations.\n\n4. Atakule Tower: This 410-foot tall tower offers panoramic views of Ankara and its surrounding mountains. It also has a revolving restaurant at the top, making it a popular spot for a romantic dinner.\n\n5. Gençlik Park: This park is a popular recreational spot 

### Sequential Chain

In [30]:
capital_template = PromptTemplate(input_variables=['country'],
                               template = "Tell me the capital of {country}")
capital_chain = LLMChain(llm = llm, prompt = capital_template, output_key="capital")

In [31]:
famous_template = PromptTemplate(input_variables=['capital'],
                                 template = "Give me some nice places to visit in {capital}")
famous_chain = LLMChain(llm = llm, prompt= famous_template,output_key="places")

In [33]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains = [capital_chain,famous_chain],
                        input_variables=['country'],
                        output_variables=['capital','places'])

In [34]:
chain({'country':"Turkey"})

/Users/tolgadincer/Documents/AI_Projects/langchain/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'country': 'Turkey',
 'capital': '\n\nAnkara',
 'places': ", Turkey:\n\n1. Anıtkabir (Atatürk's Mausoleum)\n2. Ulus Square\n3. Ankara Citadel\n4. Kızılay Square\n5. Anadolu Medeniyetleri Müzesi (Museum of Anatolian Civilizations)\n6. Atakule Tower\n7. Eymir Lake\n8. Kocatepe Mosque\n9. Hamamönü Neighborhood\n10. Gençlik Parkı (Youth Park)\n11. Aslanhane Mosque\n12. Hacı Bayram Mosque and Complex\n13. Ankara Ethnography Museum\n14. Museum of Anatolian Civilizations\n15. Ankara Castle\n16. Anka Mall\n17. Ankara Zoo\n18. Karum Shopping Center\n19. Roman Baths of Ankara\n20. Armada Shopping and Entertainment Center."}

### Chatmodels with OpenAI

In [35]:
from langchain.chat_models import ChatOpenAI

In [36]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [37]:
chatllm = ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"], temperature = 0.6,model = 'gpt-3.5-turbo')

/Users/tolgadincer/Documents/AI_Projects/langchain/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [40]:
chatllm([
    SystemMessage(content="I am Batman and you are an AI assistant for me"),
    HumanMessage(content = "Give me the current statuses of Joker and Bane")
])

AIMessage(content='As of my last update, Joker and Bane are both currently incarcerated in Arkham Asylum. Joker is being held in a maximum-security cell due to his unpredictable and dangerous nature. Bane, on the other hand, is being closely monitored due to his physical strength and intelligence. Both villains are considered to be high-risk individuals and require constant supervision.')

### Prompt Template + LLM + Output Parsers

In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser


In [42]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [44]:
template = "You are a helpful assistant. When the user gives any input, you should generate 5 words synonyms in a comma separated list"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system",template),
    ("human", human_template)
])

In [45]:
chain = chatprompt|chatllm|Commaseperatedoutput()

In [46]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' brilliant', ' sharp', ' astute']